#### Entrega - Modelo e Previsão

> Explique como você faria a previsão da nota do imdb a partir dos dados. Quais variáveis e/ou suas transformações você utilizou e por quê? Qual tipo de problema estamos resolvendo (regressão, classificação)? Qual modelo melhor se aproxima dos dados e quais seus prós e contras? Qual medida de performance do modelo foi escolhida e por quê?

Primeiramente, esse é um problema de regressão, estamos prevendo o valor de uma variável numérica (IMDB_Rating) a partir das variáveis independentes. Considerando que o dataset tem valores limitados de IMDB_Rating, por causa de algum filtro ou da origem do dataset, esse poderia também ser um problema de classificação ao criar categorías de baixo, médio ou alto rating IMDB, mas irei continuar como um problema de regressão.

Para fazer a previsão irei utilizar os dados e aplicar transformações e feature engineering para obter o máximo de informação possível dos dados, por exemplo:
- A partir do Overview pude utilizar um processo de NLP (NLTK + CountVectorizer) para transformar o em embeddings e conseguir as 300 keywords e com isso ter uma nova feature de No_keywords no dataset.
- Com Overview também criei a feature Overview_length, que é a quantidade total de palavras do Overview.
- Transformei Released_Year -> Released_Year_Group, pegando a divisão inteira por 10 e multiplicando por 10 para agrupar o ano de lançamento em agrupamentos de 10 em 10 anos.
- Separei o Runtime para Runtime_time e Runtime_category, como só temos filmes com o Runtime em minutos, posso descartar essa feature, com Runtime_time obtive o Runtime_category, com filmes short, medium ou long duration.
- Com genre irei aplicar uma transformação a depender do algoritmo que estiver usando, como Count ou Target Encoding, mas também criei uma feature No_Genres que conta o número de genêros no filme.
- Director também irei precisar aplicar uma transformação, como o Label Encodding por exemplo, mas também utilizei para criar a feature Director_No_Movies que conta quantos filmes o diretor produziu.

Em relação as outras variáveis númericas irei testar transformações como normalização e escalonamento.

Sobre os modelos e métricas irei descorrer melhor sobre isso no relatório completo (PDF)


In [26]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, MultiLabelBinarizer, FunctionTransformer, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn import metrics
from sklearn.impute import SimpleImputer


dataset =  pd.read_csv('./desafio_indicium_imdb_after_process.csv', index_col=0, sep=';')
dataset.head(3)
dataset.isna().sum()

print(dataset.groupby('Certificate').size())

Certificate
16            1
A           196
Approved     11
G            12
PG           39
PG-13        43
Passed       34
R           146
TV-14         1
TV-MA         1
TV-PG         3
U           233
UA          176
Unrated     102
dtype: int64


In [ ]:
string_var = ['Series_Title', 'Overview']
cat_var = ['Certificate', 'Genre', 'Director', 'Star1', 'Star2', 'Star3', 'Star4']
numeric_var = ['Released_Year', 'Runtime_time', 'Meta_score', 'No_of_Votes','Gross', 'No_keywords', 'Released_Year_Group', 'No_Genres', 'Director_No_Movies', 'Overview_length']

X = dataset[string_var + cat_var + numeric_var]
y = dataset['IMDB_Rating']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [35]:
# Basemodel com Lasso Regression (L1)
# Multi-label Binarizer para Genres and Actors.
# Ordinal Encoder para Certificate and Director.

from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split


X_train_l = X_train.copy()
X_test_l = X_test.copy()
y_train_l = y_train.copy()
y_test_l = y_test.copy()

X_train_l['Actors'] = X_train_l[['Star1', 'Star2', 'Star3', 'Star4']].fillna("Unknown").agg(', '.join, axis=1)
X_test_l['Actors'] = X_test_l[['Star1', 'Star2', 'Star3', 'Star4']].fillna("Unknown").agg(', '.join, axis=1)

multilabel_binarizer_genres = MultiLabelBinarizer()
X_train_l['Genre'] = multilabel_binarizer_genres.fit_transform(X_train_l['Genre'].str.split(', '))
X_test_l['Genre'] = multilabel_binarizer_genres.transform(X_test_l['Genre'].str.split(', '))

multilabel_binarizer_actors = MultiLabelBinarizer()
X_train_l['Actors'] = multilabel_binarizer_actors.fit_transform(
    X_train_l[['Star1', 'Star2', 'Star3', 'Star4']].fillna("Unknown").values.tolist()
)
X_test_l['Actors'] = multilabel_binarizer_actors.transform(
     X_test_l[['Star1', 'Star2', 'Star3', 'Star4']].fillna("Unknown").values.tolist()
)

num_pipeline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]
)

preprocessor = ColumnTransformer(transformers=[
    ('cat_var', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), [cat_var[0],cat_var[2]]),
    ('num_var', num_pipeline, numeric_var)
])

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('lasso', LassoCV(max_iter=250, cv=5, n_jobs=-1, random_state=42))
])

pipeline.fit(X_train_l, y_train_l)

y_pred = pipeline.predict(X_test_l)

r2 = metrics.r2_score(y_test_l, y_pred)
rmse = metrics.root_mean_squared_log_error(y_test_l, y_pred)

print(f'R2 score : {r2:.2f} \nRMSE : {rmse}')


R2 score : 0.35 
RMSE : 0.02435149060862094


e:\Desenvolvimento\Pessoal\indicium-2025\.venv\Lib\site-packages\sklearn\preprocessing\_label.py:909: UserWarning: unknown class(es) ['Aamir Bashir', 'Aaron Taylor-Johnson', 'Abdel Ahmed Ghili', 'Abhay Deol', 'Abigail Breslin', 'Abraham Attah', 'Adam Baldwin', 'Adam Driver', 'Adesh Prasad', 'Adil Hussain', 'Adolphe Menjou', 'Adrian Molina', 'Adrien Brody', 'Adèle Haenel', 'Agnes Moorehead', 'Ahmet Mümtaz Taylan', 'Ahna Capri', 'Ahney Her', 'Aitana Sánchez-Gijón', 'Ajay Devgn', 'Ajay Naidu', 'Akemi Yamaguchi', 'Aki Maeda', 'Akihiro Miwa', 'Akio Ôtsuka', 'Akira Ishihama', 'Akira Terao', 'Akshaye Khanna', 'Al Ernest Garcia', 'Alain Delon', 'Alakina Mann', 'Alan Alda', 'Alan Badel', 'Alan Mak', 'Alan Randolph Scott', 'Alan Ruck', 'Alan Tudyk', 'Albert Finney', 'Albert Rémy', 'Alberto Ammann', 'Aldo Giuffrè', 'Aldo Silvani', 'Alec Baldwin', 'Alec Cawthorne', 'Alec Guinness', 'Alec Secareanu', 'Alejandro Jodorowsky', 'Aleksandr Kaydanovskiy', 'Aleksey Kravchenko', 'Aleksey Rozin', 'Aleksey S